In [ ]:
import os

import glob
import wget

ROOT = "/content/drive/MyDrive/Credit-Card-Fraud-Detetion-Dataset"
data_dir = os.path.join(ROOT,'data')
os.makedirs(data_dir, exist_ok=True)

from omegaconf import OmegaConf
MODEL_CONFIG = os.path.join(data_dir,'diar_infer_telephonic.yaml')
if not os.path.exists(MODEL_CONFIG):
    config_url = "https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/diar_infer_telephonic.yaml"
    MODEL_CONFIG = wget.download(config_url,data_dir)

config = OmegaConf.load(MODEL_CONFIG)
print(OmegaConf.to_yaml(config))

output_dir = os.path.join(ROOT, 'outputs')
os.makedirs(output_dir,exist_ok=True)




#os.makedirs(data_dir, exist_ok=True)
#an4_audio = os.path.join(data_dir,'an4_diarize_test.wav')
#an4_audio = "/content/drive/MyDrive/Credit-Card-Fraud-Detetion-Dataset/day1_consultation01.wav"
#an4_rttm = os.path.join(data_dir,'an4_diarize_test.rttm')
#if not os.path.exists(an4_audio):
#    an4_audio_url = "https://nemo-public.s3.us-east-2.amazonaws.com/an4_diarize_test.wav"
#    an4_audio = wget.download(an4_audio_url, data_dir)
#if not os.path.exists(an4_rttm):
#    an4_rttm_url = "https://nemo-public.s3.us-east-2.amazonaws.com/an4_diarize_test.rttm"
#    an4_rttm = wget.download(an4_rttm_url, data_dir)



def diarize_files(an4_audio):
  print("############################################",an4_audio,"####################################################")
  # Create a manifest for input with below format.
  # {'audio_filepath': /path/to/audio_file, 'offset': 0, 'duration':None, 'label': 'infer', 'text': '-',
  # 'num_speakers': None, 'rttm_filepath': /path/to/rttm/file, 'uem_filepath'='/path/to/uem/filepath'}
  import json
  meta = {
      'audio_filepath': an4_audio,
      'offset': 0,
      'duration':None,
      'label': 'infer',
      'text': '-',
      'num_speakers': 2,
      'rttm_filepath': None,
      'uem_filepath' : None
  }
  with open(data_dir+'/input_manifest.json','w') as fp:
      json.dump(meta,fp)
      fp.write('\n')

  #!cat data/input_manifest.json

  #output_dir = os.path.join(ROOT, 'oracle_vad')
  #os.makedirs(output_dir,exist_ok=True)




  config.num_workers = 1 # Workaround for multiprocessing hanging with ipython issue

  #output_dir = os.path.join(ROOT, 'outputs')
  #os.makedirs(output_dir,exist_ok=True)
  config.diarizer.manifest_filepath = data_dir+'/input_manifest.json'
  config.diarizer.out_dir = output_dir #Directory to store intermediate files and prediction outputs

  pretrained_vad = 'vad_multilingual_marblenet'
  pretrained_speaker_model = 'titanet_large'

  config.diarizer.speaker_embeddings.model_path = pretrained_speaker_model
  config.diarizer.oracle_vad = False # compute VAD provided with model_path to vad config
  config.diarizer.clustering.parameters.oracle_num_speakers=False

  # Here, we use our in-house pretrained NeMo VAD model
  config.diarizer.vad.model_path = pretrained_vad
  config.diarizer.vad.parameters.onset = 0.8
  config.diarizer.vad.parameters.offset = 0.6
  config.diarizer.vad.parameters.pad_offset = -0.05




  from nemo.collections.asr.models import ClusteringDiarizer
  sd_model = ClusteringDiarizer(cfg=config)

  sd_model.diarize()


ROOT_mod = ROOT + "/*.wav"
wav_files = glob.glob(ROOT_mod)

for wav_file in wav_files:
  diarize_files(wav_file)
